## LSTM

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from headers import *
from spacy.vocab import Vocab
from spacy.language import Language
import spacy



In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from keras.layers.recurrent import LSTM
from keras import optimizers
import tensorflow as tf

In [ ]:
%run 'utils.ipynb'



In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(DEF_LEN, WV_SIZE)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(WV_SIZE, kernel_initializer='glorot_normal'))
model.add(Activation('softmax'))
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
plot_model(model, to_file='lstm.png', show_shapes=True)

In [ ]:
adam=optimizers.Adam(lr=3e-3)

model.compile(loss=sq_distance, optimizer=adam)


In [ ]:
model.fit(X_train[:30000],Y_train[:30000],batch_size=100,epochs=5,validation_data=(X_val,Y_val),verbose = 1)

In [ ]:
model.evaluate(x=X_test,y=Y_test,batch_size=100, verbose=1)

In [ ]:
a=model.predict(X_test[45,:,:].reshape(1,DEF_LEN,WV_SIZE), batch_size=1, verbose=1)

In [ ]:
get_most_similar(a)[:5]

In [ ]:
def get_most_similar_for_lstm(input_embedding):
    cs_list=[]
    for v in global_vocab:
        doc=nlp(v)
        cs=cosine_sim(input_embedding,doc[0].vector)
        cs_list.append((doc[0].text,cs))
    sorted_cs_list=sorted(cs_list,key=lambda tup:tup[1],reverse=True)
    return sorted_cs_list

In [ ]:
sample=dict2[-100:]
test_words=[x for x,y in sample]
ranks=[]
X_test_embed=np.zeros((100,DEF_LEN,WV_SIZE))
X_test_embed[:,:,:]=X_test[-100:,:,:]

predictions=model.predict(X_test[-100:,:,:],batch_size=10,verbose=0)
print "Done"


In [ ]:
for i in range(len(predictions)):
    input_embedding=predictions[i]
    
    r=get_list_from_tuple(get_most_similar(input_embedding))
    results=r[0]
    scores=r[1]
    target_word=Y_GT[-100+i]
    print dict2[-100+i][1],' ',target_word,'/',results[0],' ',scores[0]
    if target_word in results:
        target_rank=results.index(target_word)
        ranks.append(target_rank)
        print "%d/%d done for word %s, rank: %d"%(i,len(test_words),str(target_word),target_rank)
print "Mean rank: %f"%(float(sum(ranks))/float(len(ranks)))
print get_rank_bins(ranks)